In [160]:
import numpy as np
import pandas as pd 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

dt = pd.read_excel(r"C:\Users\utente\OneDrive\Desktop\machine learning\materiale esame\default of credit card clients.xls",index_col=0, header=1)

In [2]:
seed=589667
Feat=dt.drop(['default payment next month'], axis=1)
Tar=dt['default payment next month']
X=Feat.to_numpy()
Y=Tar.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = seed)
y_train=y_train.astype('int')
y_test=y_test.astype('int')
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:',  y_test.shape)

Training Features Shape: (24000, 23)
Training Labels Shape: (24000,)
Testing Features Shape: (6000, 23)
Testing Labels Shape: (6000,)


In [33]:
class Node:
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature=feature
        self.threshold=threshold
        self.data_left=data_left
        self.data_right=data_right
        self.gain=gain
        self.value=value

In [ ]:
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=5):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None
        
    @staticmethod
    def _entropy(s):

        counts = np.bincount(np.array(s, dtype=np.int64))
        percentages = counts / len(s)
        
        entropy = 0
        for pct in percentages:
            if pct > 0:
                entropy += pct * np.log2(pct)
        return -entropy
    
    def _information_gain(self, parent, left_child, right_child):
        num_left = len(left_child) / len(parent)
        num_right = len(right_child) / len(parent)
        
        # One-liner which implements the previously discussed formula
        return self._entropy(parent) - (num_left * self._entropy(left_child) + num_right * self._entropy(right_child))
    
    def _best_split(self, X, y):
        best_split = {}
        best_info_gain = -1
        n_rows, n_cols = X.shape
        
        for f_idx in range(n_cols):
            X_curr = X[:, f_idx]
            
            for threshold in np.unique(X_curr):
                df = np.concatenate((X, y.reshape(1, -1).T), axis=1)
                df_left = np.array([row for row in df if row[f_idx] <= threshold])
                df_right = np.array([row for row in df if row[f_idx] > threshold])
                
                if len(df_left) > 0 and len(df_right) > 0:

                    y = df[:, -1]
                    y_left = df_left[:, -1]
                    y_right = df_right[:, -1]

                    gain = self._information_gain(y, y_left, y_right)
                    if gain > best_info_gain:
                        best_split = {
                            'feature_index': f_idx,
                            'threshold': threshold,
                            'df_left': df_left,
                            'df_right': df_right,
                            'gain': gain
                        }
                        best_info_gain = gain
        return best_split
    
    def _build(self, X, y, depth=0):
        n_rows, n_cols = X.shape
        
        if n_rows >= self.min_samples_split and depth <= self.max_depth:
            best = self._best_split(X, y)
            if best['gain'] > 0:
                left = self._build(
                    X=best['df_left'][:, :-1], 
                    y=best['df_left'][:, -1], 
                    depth=depth + 1
                )
                right = self._build(
                    X=best['df_right'][:, :-1], 
                    y=best['df_right'][:, -1], 
                    depth=depth + 1
                ) 
                print('nodo ok')
                return Node(
                    feature=best['feature_index'], 
                    threshold=best['threshold'], 
                    data_left=left, 
                    data_right=right, 
                    gain=best['gain']
                )
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        self.root = self._build(X, y)
        
    def _predict(self, x, tree):

        if tree.value != None:
            return tree.value
        feature_value = x[tree.feature]
        
        if feature_value <= tree.threshold:
            return self._predict(x=x, tree=tree.data_left)
        
        if feature_value > tree.threshold:
            return self._predict(x=x, tree=tree.data_right)
        
    def predict(self, X):
        return [self._predict(x, self.root) for x in X]

In [195]:
class RandomForest:
    
    def __init__(self, num_trees=5, min_samples_split=2, max_depth=5):
        self.num_trees = num_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.decision_trees = []
        
    @staticmethod
    def _sample(X, y):
        
        n_rows, n_cols = X.shape
        samples_e = np.random.choice(a=n_rows, size=round(0.8*n_rows), replace=True)
        samples_f = np.random.choice(a=n_cols, size=round(np.sqrt(n_cols)), replace=False)
        return X[samples_e, :][:,samples_f], y[samples_e]
        
    def fit(self, X, y):
        
        if len(self.decision_trees) > 0:
            self.decision_trees = []
            
        num_built = 0
        while num_built < self.num_trees:
            try:
                clf = DecisionTree(
                    min_samples_split=self.min_samples_split,
                    max_depth=self.max_depth
                )
                _X, _y = self._sample(X, y)
                clf.fit(_X, _y)
                self.decision_trees.append(clf)
                print('albero ok')
                num_built += 1
            except Exception as e:
                continue
    
    def predict(self, X):
        y = []
        for tree in self.decision_trees:
            y.append(tree.predict(X))
        y = np.swapaxes(a=y, axis1=0, axis2=1)
        predictions = []
        for preds in y:
            counter = Counter(preds)
            predictions.append(counter.most_common(1)[0][0])
        return predictions


In [199]:
forest=RandomForest()
forest.fit(X_train, y_train)

albero ok
albero ok
albero ok
albero ok
albero ok


In [201]:
preds=forest.predict(X_test)
accuracy_score(y_test, preds)

0.7786666666666666

In [23]:
from sklearn.datasets import load_iris

iris = load_iris()
Xir = iris['data']
yir = iris['target']
X_train_ir, X_test_ir, y_train_ir, y_test_ir = train_test_split(Xir, yir, test_size=0.2, random_state=42)

In [51]:
print(len(iris['data']))

150


In [197]:
forest2=RandomForest()
forest2.fit(X_train_ir, y_train_ir)
preds=forest2.predict(X_test_ir)

albero ok
albero ok
albero ok
albero ok
albero ok


In [198]:
accuracy_score(y_test_ir, preds)

0.36666666666666664